# CFReT Buscar analysis


This notebook demonstrates applying the BUSCAR pipeline to the CFReT pilot Cell Painting dataset.
It walks through data loading, signature extraction, clustering, measuring phenotypic activity
(between a reference control and experimental treatments) and ranking treatments by effect.

Data & references
- Data source: CFReT pilot dataset (see paper: https://www.ahajournals.org/doi/full/10.1161/CIRCULATIONAHA.124.071956)
- Original data repo: https://github.com/WayScience/cellpainting_predicts_cardiac_fibrosis

In [1]:
import sys
import json
import pathlib

import polars as pl

sys.path.append("../../")
from utils.io_utils import load_profiles

# from utils.metrics import measure_phenotypic_activity
from utils.data_utils import split_meta_and_features
from utils.signatures import get_signatures
from utils.heterogeneity import optimized_clustering
from utils.metrics import measure_phenotypic_activity
from utils.identify_hits import identify_compound_hit
from utils.preprocess import apply_pca

/home/erikserrano/Software/miniconda3/envs/buscar/lib/python3.12/site-packages/louvain/__init__.py:54: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound


Setting paramters

In [2]:
# setting parameters
treatment_col = "Metadata_treatment"
treatment_heart_col = "Metadata_treatment_and_heart"

# parameters used for clustering optimization
cfret_pilot_cluster_param_grid = {
    # Clustering resolution: how granular the clusters should be
    "cluster_resolution": {"type": "float", "low": 0.1, "high": 2.2},
    # Number of neighbors for graph construction
    "n_neighbors": {"type": "int", "low": 5, "high": 100},
    # Clustering algorithm
    "cluster_method": {"type": "categorical", "choices": ["leiden"]},
    # Distance metric for neighbor computation
    "neighbor_distance_metric": {
        "type": "categorical",
        "choices": ["euclidean", "cosine", "manhattan"],
    },
}

Setting input and output paths

In [3]:
# load in raw data from
cfret_data_dir = pathlib.Path("../0.download-data/data/sc-profiles/cfret/").resolve(
    strict=True
)
cfret_profiles_path = (
    cfret_data_dir / "localhost230405150001_sc_feature_selected.parquet"
).resolve(strict=True)
cfret_feature_space_path = (
    cfret_data_dir / "cfret_feature_space_configs.json"
).resolve(strict=True)

# make results dir
results_dir = pathlib.Path("./results/cfret-pilot").resolve()
results_dir.mkdir(parents=True, exist_ok=True)

Data preprocessing 
- 

In [4]:
# loading profiles
cfret_df = load_profiles(cfret_profiles_path)

# add another metadata column that combins both Metadata_heart_number and Metadata_treatment
cfret_df = cfret_df.with_columns(
    (
        pl.col("Metadata_treatment").cast(pl.Utf8)
        + "_heart_"
        + pl.col("Metadata_heart_number").cast(pl.Utf8)
    ).alias("Metadata_treatment_and_heart")
)

# Update the Metadata_treatment column that distinguishes what is the reference
# among other treatments in this example we are using heart 11 + DMSO as our
# reference (healthy heart + DMSO)
cfret_df = cfret_df.with_columns(
    pl.when(
        (pl.col("Metadata_treatment") == "DMSO")
        & (pl.col("Metadata_heart_number") == 11)
    )
    .then(pl.lit("DMSO_heart_11"))
    .otherwise(pl.col("Metadata_treatment"))
    .alias("Metadata_treatment")
)
cfret_df = cfret_df.with_columns(
    pl.when(
        (pl.col("Metadata_treatment") == "DMSO")
        & (pl.col("Metadata_heart_number") == 9)
    )
    .then(pl.lit("DMSO_heart_9"))
    .otherwise(pl.col("Metadata_treatment"))
    .alias("Metadata_treatment")
)

# split features
cfret_meta, cfret_feats = split_meta_and_features(cfret_df)


# Display data
cfret_df.head()

Metadata_cell_id,Metadata_WellRow,Metadata_WellCol,Metadata_heart_number,Metadata_cell_type,Metadata_heart_failure_type,Metadata_treatment,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Cells_Location_Center_X,Metadata_Cells_Location_Center_Y,Metadata_Image_Count_Cells,Metadata_ImageNumber,Metadata_Plate,Metadata_Well,Metadata_Cells_Number_Object_Number,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,Metadata_Nuclei_Number_Object_Number,Metadata_Site,Cytoplasm_AreaShape_BoundingBoxMinimum_X,Cytoplasm_AreaShape_Compactness,Cytoplasm_AreaShape_Eccentricity,Cytoplasm_AreaShape_Extent,Cytoplasm_AreaShape_FormFactor,Cytoplasm_AreaShape_MajorAxisLength,Cytoplasm_AreaShape_MeanRadius,Cytoplasm_AreaShape_MinorAxisLength,Cytoplasm_AreaShape_Perimeter,Cytoplasm_AreaShape_Solidity,Cytoplasm_AreaShape_Zernike_0_0,Cytoplasm_AreaShape_Zernike_1_1,Cytoplasm_AreaShape_Zernike_2_0,Cytoplasm_AreaShape_Zernike_2_2,Cytoplasm_AreaShape_Zernike_3_1,Cytoplasm_AreaShape_Zernike_4_0,Cytoplasm_AreaShape_Zernike_4_2,…,Nuclei_Texture_DifferenceVariance_Actin_3_01_256,Nuclei_Texture_DifferenceVariance_Mitochondria_3_03_256,Nuclei_Texture_DifferenceVariance_PM_3_03_256,Nuclei_Texture_InfoMeas1_ER_3_00_256,Nuclei_Texture_InfoMeas1_ER_3_01_256,Nuclei_Texture_InfoMeas1_ER_3_02_256,Nuclei_Texture_InfoMeas1_ER_3_03_256,Nuclei_Texture_InfoMeas1_Hoechst_3_00_256,Nuclei_Texture_InfoMeas1_Hoechst_3_01_256,Nuclei_Texture_InfoMeas1_Hoechst_3_02_256,Nuclei_Texture_InfoMeas1_Hoechst_3_03_256,Nuclei_Texture_InfoMeas1_Mitochondria_3_00_256,Nuclei_Texture_InfoMeas1_Mitochondria_3_01_256,Nuclei_Texture_InfoMeas1_Mitochondria_3_02_256,Nuclei_Texture_InfoMeas1_Mitochondria_3_03_256,Nuclei_Texture_InfoMeas1_PM_3_00_256,Nuclei_Texture_InfoMeas1_PM_3_01_256,Nuclei_Texture_InfoMeas1_PM_3_02_256,Nuclei_Texture_InfoMeas1_PM_3_03_256,Nuclei_Texture_InfoMeas2_ER_3_01_256,Nuclei_Texture_InfoMeas2_ER_3_03_256,Nuclei_Texture_InfoMeas2_Hoechst_3_01_256,Nuclei_Texture_InfoMeas2_Hoechst_3_03_256,Nuclei_Texture_InfoMeas2_PM_3_01_256,Nuclei_Texture_InfoMeas2_PM_3_03_256,Nuclei_Texture_InverseDifferenceMoment_Actin_3_02_256,Nuclei_Texture_InverseDifferenceMoment_ER_3_01_256,Nuclei_Texture_InverseDifferenceMoment_ER_3_03_256,Nuclei_Texture_InverseDifferenceMoment_Mitochondria_3_03_256,Nuclei_Texture_InverseDifferenceMoment_PM_3_01_256,Nuclei_Texture_InverseDifferenceMoment_PM_3_03_256,Nuclei_Texture_SumEntropy_PM_3_01_256,Nuclei_Texture_SumVariance_ER_3_03_256,Nuclei_Texture_SumVariance_Hoechst_3_03_256,Nuclei_Texture_SumVariance_Mitochondria_3_01_256,Nuclei_Texture_SumVariance_PM_3_01_256,Metadata_treatment_and_heart
str,str,i64,i64,str,str,str,f64,f64,f64,f64,i64,i64,str,str,i64,i64,i64,i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
"""210e4376efde9d557a5c60029bdda6…","""B""",2,9,"""failing""","""rejected""","""DMSO_heart_9""",221.046761,137.115493,246.6028,109.285755,40,1,"""localhost230405150001""","""B02""",1,1,6,6,"""f00""",-1.35494,0.841229,0.648883,-0.850138,-1.045214,1.298358,0.376165,0.935101,1.530228,-0.983617,-0.261031,-0.299817,-0.721977,0.944725,0.161074,0.532329,1.845864,…,-0.052719,0.797095,0.359081,-0.173336,0.300041,0.217945,-0.039774,0.488531,0.472164,0.28659,0.464359,0.501649,0.507623,1.076663,0.741941,-0.696022,-0.178762,0.186741,0.158222,0.341595,0.50487,-0.440604,-0.426966,0.194372,-0.035117,0.400021,-0.619206,-0.393448,0.961214,0.406068,0.374039,-0.280532,-0.158967,-0.344804,-0.263653,-0.305486,"""DMSO_heart_9"""
"""cef18f209640ef8ae98ec110cfdcb6…","""B""",2,9,"""failing""","""rejected""","""DMSO_heart_9""",690.596142,183.067828,716.170091,177.132195,40,1,"""localhost230405150001""","""B02""",2,2,7,7,"""f00""",0.657107,-0.850399,-0.584931,2.090925,1.263259,-0.021031,1.627957,0.944161,-0.085511,1.475345,2.164761,-0.688462,1.215015,1.499086,-0.77066

In [5]:
# show how many cells per treatment
# shows the number of cells per treatment that will be clustered.
cells_per_treatment_counts = (
    cfret_df.group_by(treatment_heart_col).count().sort(treatment_heart_col)
)
cells_per_treatment_counts

/tmp/ipykernel_339079/1240663315.py:4: DeprecationWarning: `GroupBy.count` was renamed; use `GroupBy.len` instead
  cfret_df.group_by(treatment_heart_col).count().sort(treatment_heart_col)


Metadata_treatment_and_heart,count
str,u32
"""DMSO_heart_11""",1366
"""DMSO_heart_9""",9153
"""TGFRi_heart_11""",1486
"""TGFRi_heart_9""",3788
"""drug_x_heart_11""",1427
"""drug_x_heart_9""",3645


## BUSCAR pipeline

Creating on and off morphology signatures

In [6]:
# setting output paths
signatures_outpath = (results_dir / "cfret_pilot_signatures.json").resolve()

if signatures_outpath.exists():
    print("Signatures already exist, skipping this step.")
    with open(signatures_outpath, "r") as f:
        sigs = json.load(f)
        on_sigs = sigs["on"]
        off_sigs = sigs["off"]
else:
    # once the data is loaded, separate the controls
    negcon_df = cfret_df.filter(pl.col("Metadata_treatment") == "DMSO_heart_9")
    poscon_df = cfret_df.filter(pl.col("Metadata_treatment") == "DMSO_heart_11")

    # creating signatures
    on_sigs, off_sigs, _ = get_signatures(
        ref_profiles=negcon_df,
        exp_profiles=poscon_df,
        morph_feats=cfret_feats,
        test_method="mann_whitney_u",
    )

    # Save signatures as json file
    with open(signatures_outpath, "w") as f:
        json.dump({"on": on_sigs, "off": off_sigs}, f, indent=4)

Search for heterogenous effects for each treatment

In [7]:
# Apply PCA to cfret_data
pca_cfret_df = apply_pca(
    profiles=cfret_df,
    meta_features=cfret_meta,
    morph_features=cfret_feats,
    var_explained=0.95,
    seed=0,
)

# update cfret_feats because PCA was applied
cfret_pca_feats = pca_cfret_df.drop(cfret_meta).columns

In [ ]:
# number of cores required for optuna jobs
# each optuna jobs is distrubred per treament
cfret_n_jobs = cfret_df["Metadata_treatment_and_heart"].n_unique()
print(f"Number of unique treatments (hearts + treatment): {cfret_n_jobs}")

# check if the cluster labels already exist; if so just load the labels and skip optimization
# if not run optimization
cluster_labels_output = (results_dir / "cfret_pilot_cluster_labels.parquet").resolve()
if cluster_labels_output.exists():
    print("Cluster labels already exist, skipping clustering optimization.")
    cfret_cluster_labels_df = pl.read_parquet(cluster_labels_output)
    cfret_best_params = (
        None  # You may want to load best params from a saved file if needed
    )
else:
    # optimizing clustering
    cfret_cluster_labels_df, cfret_best_params = optimized_clustering(
        profiles=pca_cfret_df,
        meta_features=cfret_meta,
        morph_features=cfret_pca_feats,
        treatment_col=treatment_heart_col,
        param_grid=cfret_pilot_cluster_param_grid,
        n_trials=500,
        n_jobs=cfret_n_jobs,
        seed=0,
        study_name="cfret_pilot_pca",
    )

    # write out cluster labels
    cfret_cluster_labels_df.write_parquet(cluster_labels_output)

    # write best params as a json file
    with open(results_dir / "cfret_pilot_best_clustering_params.json", "w") as f:
        json.dump(cfret_best_params, f, indent=4)

Number of unique treatments (hearts + treatment): 6
Optimizing clustering for 6 treatment(s) with 6 job(s)...


/home/erikserrano/Software/miniconda3/envs/buscar/lib/python3.12/site-packages/louvain/__init__.py:54: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound
/home/erikserrano/Software/miniconda3/envs/buscar/lib/python3.12/site-packages/louvain/__init__.py:54: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound
/home/erikserrano/Software/miniconda3/envs/buscar/lib/python3.12/site-packages/louvain/__init__.py:54: UserWarning: pkg_resources is deprecated as an API. See https://setuptool

  DMSO_heart_9: silhouette=-0.001, params={'cluster_resolution': 1.2885634587473769, 'n_neighbors': 14, 'cluster_method': 'leiden', 'neighbor_distance_metric': 'manhattan'}
  drug_x_heart_9: silhouette=0.006, params={'cluster_resolution': 1.1850800391535194, 'n_neighbors': 30, 'cluster_method': 'leiden', 'neighbor_distance_metric': 'manhattan'}
  TGFRi_heart_9: silhouette=0.004, params={'cluster_resolution': 2.0546310508099337, 'n_neighbors': 57, 'cluster_method': 'leiden', 'neighbor_distance_metric': 'euclidean'}
  DMSO_heart_11: silhouette=0.011, params={'cluster_resolution': 1.808553434505587, 'n_neighbors': 94, 'cluster_method': 'leiden', 'neighbor_distance_metric': 'cosine'}
  drug_x_heart_11: silhouette=0.203, params={'cluster_resolution': 0.26087142409254793, 'n_neighbors': 20, 'cluster_method': 'leiden', 'neighbor_distance_metric': 'cosine'}
  TGFRi_heart_11: silhouette=0.041, params={'cluster_resolution': 0.7747165384593787, 'n_neighbors': 43, 'cluster_method': 'leiden', 'neig

sys:1: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance


In [ ]:
# merge cfret_df with the cluster labels and make sure to drop duplicate Metadata_cell_id columns
labeled_cfret_df = cfret_df.join(
    cfret_cluster_labels_df,
    on=["Metadata_cell_id"],
    how="left",
)

# check if the no rows added
if cfret_df.height != labeled_cfret_df.height:
    raise ValueError("Merged DataFrame has different number of rows!")

In [ ]:
# setting output paths
treatment_dist_scores_outpath = (
    results_dir / "treatment_phenotypic_scores.csv"
).resolve()
if treatment_dist_scores_outpath.exists():
    print("Treatment phenotypic distance scores already exist, skipping this step.")
    treatment_heart_dist_scores = pl.read_csv(treatment_dist_scores_outpath)

else:
    treatment_heart_dist_scores = measure_phenotypic_activity(
        profiles=labeled_cfret_df,
        on_signature=on_sigs,
        off_signature=off_sigs,
        ref_treatment="DMSO_heart_11",
        treatment_col=treatment_heart_col,
    )

    # save those as csv files
    treatment_heart_dist_scores.write_csv(treatment_dist_scores_outpath)

In [12]:
treatment_heart_dist_scores

ref_cluster,treatment,trt_cluster,on_dist,off_dist,exp_cluster_ratio
str,str,str,f64,f64,f64
"""DMSO_heart_11_leiden_3""","""TGFRi_heart_9""","""TGFRi_heart_9_leiden_1""",29.882965,11.252678,0.091077
"""DMSO_heart_11_leiden_6""","""TGFRi_heart_9""","""TGFRi_heart_9_leiden_1""",25.31051,11.226979,0.091077
"""DMSO_heart_11_leiden_1""","""TGFRi_heart_9""","""TGFRi_heart_9_leiden_1""",30.730772,11.38931,0.091077
"""DMSO_heart_11_leiden_7""","""TGFRi_heart_9""","""TGFRi_heart_9_leiden_1""",28.194008,10.765977,0.091077
"""DMSO_heart_11_leiden_10""","""TGFRi_heart_9""","""TGFRi_heart_9_leiden_1""",28.507084,11.278146,0.091077
…,…,…,…,…,…
"""DMSO_heart_11_leiden_11""","""DMSO_heart_9""","""DMSO_heart_9_leiden_11""",34.033759,12.921012,0.022506
"""DMSO_heart_11_leiden_5""","""DMSO_heart_9""","""DMSO_heart_9_leiden_11""",33.117793,12.016839,0.022506
"""DMSO_heart_11_leiden_9""","""DMSO_heart_9""","""DMSO_heart_9_leiden_11""",39.263147,12.84764,0.022506


In [13]:
# setting outptut paths
treatment_heart_rankings_outpath = (
    results_dir / "treatment_heart_rankings.csv"
).resolve()
if treatment_heart_rankings_outpath.exists():
    print("Treatment heart rankings already exist, skipping this step.")
    treatment_heart_rankings = pl.read_csv(treatment_heart_rankings_outpath)
else:
    treatment_heart_rankings = identify_compound_hit(
        distance_df=treatment_heart_dist_scores, method="weighted_sum"
    )

    # save as csv files
    treatment_heart_rankings.write_csv(treatment_heart_rankings_outpath)